<a href="https://colab.research.google.com/github/mesuvein23/datascience/blob/master/employees_sampling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd

# df =pd.read_csv('')
df =pd.read_excel('/content/Employees.xlsx')

df

,No,First Name,Last Name,Gender,Start Date,Years,Department,Country,Center,Monthly Salary,Annual Salary,Job Rate,Sick Leaves,Unpaid Leaves,Overtime Hours
0,1,Ghadir,Hmshw,Male,2018-04-04,2,Quality Control,Egypt,West,1560,18720,3.0,1,0,183
1,2,Omar,Hishan,Male,2020-05-21,0,Quality Control,Saudi Arabia,West,3247,38964,1.0,0,5,198
2,3,Ailya,Sharaf,Female,2017-09-28,3,Major Mfg Projects,Saudi Arabia,West,2506,30072,2.0,0,3,192
3,4,Lwiy,Qbany,Male,2018-08-14,2,Manufacturing,United Arab Emirates,Main,1828,21936,3.0,0,0,7
4,5,Ahmad,Bikri,Male,2020-03-11,0,Manufacturing,Egypt,Main,970,11640,5.0,0,5,121
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
684,685,Sari,Hanna,Male,2020-05-26,0,Marketing,Lebanon,South,1452,17424,2.0,0,3,1
685,686,Eubayda,Kayd,Male,2020-06-03,0,Facilities/Engineering,Egypt,North,3237,38844,3.0,1,0,4
686,687,Khalil,Alkalu,Male,2017-07-11,3,Facilities/Engineering,Egypt,North,2819,33828,5.0,0,0,0
687,688,Muhamad,Shrbjy,Male,2018-05-30,2,Creative,Egypt,North,2069,24828,3.0,0,0,10


In [ ]:
# Grouping by Department to get a real categorical summary
emp_details = df.groupby('Department').agg({
    'No': 'first',               # id
    'Annual Salary': 'sum',      # Total salaryon this dept
    'Country': 'first',          # Sample country for this dept
    'Gender': 'first',           # Sample gender
    'Job Rate': 'mean'           # Average rating for dept
}).reset_index()

# Rename for clarity
emp_details.columns = ['Department', 'Person_id', 'Income', 'Location', 'Gender', 'Rating']

emp_details.head()

,Department,Person_id,Income,Location,Gender,Rating
0,Account Management,8,1952664,Egypt,Male,3.690476
1,Creative,47,458580,United Arab Emirates,Male,3.789474
2,Environmental Compliance,32,391272,Syria,Male,3.076923
3,Environmental Health/Safety,74,216048,United Arab Emirates,Male,4.222222
4,Facilities/Engineering,13,1590552,Egypt,Female,3.620690


this is an summary of employee data l

In [79]:
def summarize(df: pd.DataFrame, name: str):
    out = {
        "name": name,
        "n": len(df),
        "mean_income": df["Income"].mean(),
        "median_income": df["Income"].median(),
        "mean_rating": df["Rating"].mean(),
        "gender_dist": df["Gender"].value_counts(normalize=True).round(3).to_dict(),
        "department_dist": df["Department"].value_counts(normalize=True).round(3).to_dict(),
        "location_dist": df["Location"].value_counts(normalize=True).round(3).to_dict(),
    }
    return out

emp_summary = summarize(emp_details, "Employees")
emp_summary


{'name': 'Employees',
 'n': 20,
 'mean_income': np.float64(854994.6),
 'median_income': 456018.0,
 'mean_rating': np.float64(3.5817827759024055),
 'gender_dist': {'Male': 0.75, 'Female': 0.25},
 'department_dist': {'Account Management': 0.05,
  'Creative': 0.05,
  'Environmental Compliance': 0.05,
  'Environmental Health/Safety': 0.05,
  'Facilities/Engineering': 0.05,
  'Green Building': 0.05,
  'Human Resources': 0.05,
  'IT': 0.05,
  'Major Mfg Projects': 0.05,
  'Manufacturing': 0.05,
  'Manufacturing Admin': 0.05,
  'Marketing': 0.05,
  'Product Development': 0.05,
  'Professional Training Group': 0.05,
  'Quality Assurance': 0.05,
  'Quality Control': 0.05,
  'Research Center': 0.05,
  'Research/Development': 0.05,
  'Sales': 0.05,
  'Training': 0.05},
 'location_dist': {'Egypt': 0.4,
  'United Arab Emirates': 0.25,
  'Saudi Arabia': 0.2,
  'Syria': 0.15}}

employyes ko sabai summarry dinxa yesle average income,age and distribustion of age

samling techniques


In [80]:
from typing import Dict, List, Tuple

def simple_random_sampling(df: pd.DataFrame, n: int, seed: int = 42) -> pd.DataFrame:
    return df.sample(n=n, random_state=seed).reset_index(drop=True)

def systematic_sampling(df: pd.DataFrame, n: int, seed: int = 42) -> pd.DataFrame:
    # Shuffle to avoid ordering bias, then take every k-th record
    d = df.sample(frac=1, random_state=seed).reset_index(drop=True)
    k = max(len(d) // n, 1)
    start = seed % k
    sample = d.iloc[start::k].head(n).reset_index(drop=True)
    return sample

def stratified_sampling(df: pd.DataFrame, strata_col: str, n: int, seed: int = 42) -> pd.DataFrame:
    # Proportional allocation across strata
    rng = np.random.default_rng(seed)
    counts = df[strata_col].value_counts()
    props = (counts / len(df))
    alloc = (props * n).round().astype(int)

    # Fix rounding issues to sum exactly to n
    diff = n - alloc.sum()
    if diff != 0:
        # Distribute the remaining +/- across largest strata
        order = props.sort_values(ascending=False).index.tolist()
        i = 0
        while diff != 0:
            s = order[i % len(order)]
            if diff > 0:
                alloc[s] += 1
                diff -= 1
            else:
                if alloc[s] > 0:
                    alloc[s] -= 1
                    diff += 1
            i += 1

    parts = []
    for s, k in alloc.items():
        group = df[df[strata_col] == s]
        if k > len(group):
            k = len(group)
        # Use numpy choice for reproducibility
        idx = rng.choice(group.index.to_numpy(), size=k, replace=False)
        parts.append(df.loc[idx])
    return pd.concat(parts).sample(frac=1, random_state=seed).reset_index(drop=True)

def cluster_sampling(df: pd.DataFrame, cluster_col: str, n_clusters: int, seed: int = 42) -> pd.DataFrame:
    rng = np.random.default_rng(seed)
    clusters = df[cluster_col].unique().tolist()
    chosen = rng.choice(clusters, size=min(n_clusters, len(clusters)), replace=False)
    return df[df[cluster_col].isin(chosen)].reset_index(drop=True)

# Non-probability

def convenience_sampling(df: pd.DataFrame, n: int) -> pd.DataFrame:
    # "Easy to reach" simulated as first n rows (ordered by person_id)
    return df.sort_values("Person_id").head(n).reset_index(drop=True)

def quota_sampling(df: pd.DataFrame, quotas: Dict[str, Dict[str, int]], seed: int = 42) -> pd.DataFrame:
    # quotas like {"gender": {"Male": 120, "Female": 80}}
    rng = np.random.default_rng(seed)
    pieces = []
    for col, qmap in quotas.items():
        # Apply quota independently, then union; for clean demo, we assume single quota column
        for value, k in qmap.items():
            group = df[df[col] == value]
            k = min(k, len(group))
            idx = rng.choice(group.index.to_numpy(), size=k, replace=False)
            pieces.append(df.loc[idx])
    # Remove duplicates if any and shuffle
    return pd.concat(pieces).drop_duplicates("Person_id").sample(frac=1, random_state=seed).reset_index(drop=True)

def snowball_sampling(df: pd.DataFrame, seed_ids: List[int], n: int, seed: int = 42) -> pd.DataFrame:
    # Simulate a social graph using random contacts per person
    rng = np.random.default_rng(seed)
    ids = df["Person_id"].to_numpy()
    id_to_idx = {pid: i for i, pid in enumerate(ids)}

    # Build a random adjacency list (each person connects to 3-8 others)
    adjacency = {}
    for pid in ids:
        deg = int(rng.integers(3, 9))
        neighbors = rng.choice(ids, size=deg, replace=False)
        adjacency[pid] = neighbors.tolist()

    selected = []
    queue = list(seed_ids)

    seen = set()
    while queue and len(selected) < n:
        pid = queue.pop(0)
        if pid in seen:
            continue
        seen.add(pid)
        selected.append(pid)
        # add neighbors (referrals)
        for nb in adjacency.get(pid, []):
            if nb not in seen and len(queue) < 10_000:
                queue.append(nb)

    sample = df[df["Person_id"].isin(selected)].copy()
    return sample.reset_index(drop=True)

In [ ]:
SAMPLE_N = 10

srs = simple_random_sampling(emp_details, SAMPLE_N, seed=42)
sys = systematic_sampling(emp_details, SAMPLE_N, seed=42)
strat = stratified_sampling(emp_details, "Department", SAMPLE_N, seed=42)

# Cluster sampling selects clusters; sample size will vary
cluster = cluster_sampling(emp_details, "Location", n_clusters=2, seed=42)
# cluster

conv = convenience_sampling(emp_details, SAMPLE_N)

quota = quota_sampling(
    emp_details,
    quotas={"Gender": {"Male": 275, "Female": 225}},  # force a chosen ratio
    seed=42
)

snow = snowball_sampling(emp_details, seed_ids=[10, 200, 999], n=SAMPLE_N, seed=42)

samples = {
    "Simple Random": srs,
    "Systematic": sys,
    "Stratified (by Department)": strat,
    "Cluster (2 Location)": cluster,
    "Convenience": conv,
    "Quota (Gender)": quota,
    "Snowball (simulated)": snow,
}

summaries = pd.DataFrame([summarize(df, name) for name, df in [("Employees", emp_details), *samples.items()]])
summaries


,name,n,mean_income,median_income,mean_age,gender_dist,sector_dist,city_dist
0,Employees,20,854994.6,456018.0,3.581783,"{'Male': 0.75, 'Female': 0.25}","{'Account Management': 0.05, 'Creative': 0.05,...","{'Egypt': 0.4, 'United Arab Emirates': 0.25, '..."
1,Simple Random,10,713203.2,337314.0,3.559330,"{'Male': 0.6, 'Female': 0.4}","{'Account Management': 0.1, 'Research/Developm...","{'Egypt': 0.5, 'United Arab Emirates': 0.3, 'S..."
2,Systematic,10,1400989.2,1388880.0,3.652335,"{'Male': 0.6, 'Female': 0.4}","{'Account Management': 0.1, 'Quality Control':...","{'Egypt': 0.5, 'United Arab Emirates': 0.3, 'S..."
3,Stratified (by Department),10,962179.2,424926.0,3.701764,"{'Male': 0.7, 'Female': 0.3}","{'Major Mfg Projects': 0.1, 'Creative': 0.1, '...","{'Egypt': 0.4, 'United Arab Emirates': 0.3, 'S..."
4,Cluster (2 Location),12,960496.0,908286.0,3.524455,"{'Male': 0.667, 'Female': 0.333}","{'Account Management': 0.083, 'Facilities/Engi...","{'Egypt': 0.667, 'Saudi Arabia': 0.333}"
5,Convenience,10,1299318.0,1101090.0,3.552330,"{'Male': 0.5, 'Female': 0.5}","{'Quality Control': 0.1, 'Major Mfg Projects':...","{'Egypt': 0.6, 'Saudi Arabia': 0.2, 'United Ar..."
6,Quota (Gender),20,854994.6,456018.0,3.581783,"{'Male': 0.75, 'Female': 0.25}","{'Account Management': 0.05, 'Marketing': 0.05...","{'Egypt': 0.4, 'United Arab Emirates': 0.25, '..."
7,Snowball (simulated),0,NaN,NaN,NaN,{},{},{}


In [ ]:
def dist_table(df: pd.DataFrame, col: str) -> pd.Series:
    return df[col].value_counts(normalize=True).rename("prop")

emp_gender = dist_table(emp_details, "Gender")
emp_sector = dist_table(emp_details, "Department")
emp_city   = dist_table(emp_details, "Location")

def compare_dist(col: str) -> pd.DataFrame:
    base = dist_table(emp_details, col).rename("Population")
    out = base.to_frame()
    for name, df in samples.items():
        out[name] = dist_table(df, col)
    return out.fillna(0).round(3)

gender_cmp = compare_dist("Gender")
department_cmp = compare_dist("Department")
location_cmp   = compare_dist("Location")

gender_cmp

,Population,Simple Random,Systematic,Stratified (by Department),Cluster (2 Location),Convenience,Quota (Gender),Snowball (simulated)
Gender,,,,,,,,
Male,0.75,0.6,0.6,0.7,0.667,0.5,0.75,0.0
Female,0.25,0.4,0.4,0.3,0.333,0.5,0.25,0.0


In [ ]:
department_cmp

,Population,Simple Random,Systematic,Stratified (by sector),Cluster (2 cities),Convenience,Quota (Gender),Snowball (simulated)
Department,,,,,,,,
Account Management,0.05,0.1,0.1,0.1,0.083,0.1,0.05,0.0
Creative,0.05,0.1,0.0,0.1,0.000,0.0,0.05,0.0
Environmental Compliance,0.05,0.0,0.0,0.1,0.000,0.0,0.05,0.0
Environmental Health/Safety,0.05,0.1,0.0,0.1,0.000,0.0,0.05,0.0
Facilities/Engineering,0.05,0.0,0.1,0.1,0.083,0.1,0.05,0.0
Green Building,0.05,0.1,0.0,0.1,0.083,0.1,0.05,0.0
Human Resources,0.05,0.0,0.0,0.1,0.000,0.0,0.05,0.0
IT,0.05,0.0,0.1,0.1,0.083,0.1,0.05,0.0
Major Mfg Projects,0.05,0.1,0.1,0.1,0.083,0.1,0.05,0.0


In [ ]:
location_cmp

,Population,Simple Random,Systematic,Stratified (by sector),Cluster (2 cities),Convenience,Quota (Gender),Snowball (simulated)
Location,,,,,,,,
Egypt,0.40,0.5,0.5,0.4,0.667,0.6,0.40,0.0
United Arab Emirates,0.25,0.3,0.3,0.3,0.000,0.2,0.25,0.0
Saudi Arabia,0.20,0.1,0.2,0.1,0.333,0.2,0.20,0.0
Syria,0.15,0.1,0.0,0.2,0.000,0.0,0.15,0.0


mean income error vs employees


In [ ]:
emp_mean_income = emp_details["Income"].mean()

errors = []
for name, df in samples.items():
    err = df["Income"].mean() - emp_mean_income
    errors.append({"sample": name, "n": len(df), "mean_income_error": float(err)})

err_df = pd.DataFrame(errors).sort_values("mean_income_error")
err_df

,sample,n,mean_income_error
0,Simple Random,10,-141791.4
5,Quota (Gender),20,0.0
3,Cluster (2 cities),12,105501.4
2,Stratified (by sector),10,107184.6
4,Convenience,10,444323.4
1,Systematic,10,545994.6
6,Snowball (simulated),0,NaN
